# 4 different ways to Combine two (or more) strategies to backtest on multiple assets 

Reference 0: https://github.com/polakowo/vectorbt/blob/master/tests/notebooks/indicators.ipynb

Reference 1: strategy combination: https://github.com/sergio12S/youtoube/blob/master/vectorbt/combination_parameters.ipynb

Reference 2 - create a custom indicator: https://greyhoundanalytics.com/blog/create-a-custom-indicator-in-vectorbt/

Will refer: https://www.youtube.com/watch?v=JOdEZMcvyac&t=11658s

In [1]:
import pandas as pd
import numpy
import vectorbt as vbt
from numba import njit

In [2]:
symbols = ["BTC-USD","ETH-USD"]
fast_window, slow_window, rsi_window, top, bottom = (180,240, 14, 80,20)
fees = 0.001
interval = "1D"
start = "2020-01-01"

In [3]:
data = vbt.YFData.download(symbols,
                           start=start, 
                           interval=interval).get("Close")
data

symbol,BTC-USD,ETH-USD
Date,,
2019-12-31 00:00:00+00:00,7193.599121,129.610855
2020-01-01 00:00:00+00:00,7200.174316,130.802002
2020-01-02 00:00:00+00:00,6985.470215,127.410179
2020-01-03 00:00:00+00:00,7344.884277,134.171707
2020-01-04 00:00:00+00:00,7410.656738,135.069366
...,...,...
2023-08-15 00:00:00+00:00,29170.347656,1826.932739
2023-08-16 00:00:00+00:00,28701.779297,1805.659058
2023-08-17 00:00:00+00:00,26664.550781,1684.933472


### Approach 1: using "&" operator

(Note: my preferred method)

In [4]:
rsi = vbt.RSI.run(data, window=rsi_window, short_name ="RSI")
fast = vbt.MA.run(data, window=fast_window, short_name="fast")
slow = vbt.MA.run(data, window=slow_window, short_name="slow")

entries = fast.ma_above(slow).vbt & rsi.rsi_below(top).vbt
exits = fast.ma_below(slow).vbt & rsi.rsi_above(bottom).vbt

pf = vbt.Portfolio.from_signals(data, entries, exits, fees=fees, freq=interval)
pf.total_return()

fast_window  slow_window  RSI_window  symbol 
180          240          14          BTC-USD    2.254842
                                      ETH-USD    5.267745
Name: total_return, dtype: float64

### Approach 2: Combine indicators using to_numpy()

- (a) built-in vbt indicator + numpy 

In [5]:
def get_signals(data, fast_window, slow_window, rsi_window, top, bottom):
    rsi = vbt.RSI.run(data, window=rsi_window, short_name = "RSI").rsi.to_numpy()
    fast = vbt.MA.run(data, window=fast_window, short_name="fast").ma.to_numpy()
    slow = vbt.MA.run(data, window=slow_window, short_name="slow").ma.to_numpy()
    entries = (fast > slow) & (rsi < top)
    exits = (fast < slow) & (rsi > bottom)
    return entries, exits

entries, exits = get_signals(data, fast_window, slow_window, 
                             rsi_window, top, bottom)
pf = vbt.Portfolio.from_signals(data, entries, exits, 
                                fees=fees, freq=interval)
pf.total_return()  

symbol
BTC-USD    2.254842
ETH-USD    5.267745
Name: total_return, dtype: float64

- (b) talib indicator + numpy

(Note: got accuracy problem for talib?)

In [6]:
# I try to replicate the above result with talib package but I get inconsistent result for ETH-USD
# (e.g., ETH-USD should be -0.262052 but in this example, I get -0.242040 instead)
## Possible Reason: talib RSI seems calculated in a very different way compared to others: https://github.com/TA-Lib/ta-lib-python/issues/448

def get_signals(data, fast_window, slow_window, rsi_window, top, bottom):
    rsi = vbt.talib("RSI").run(data, timeperiod=rsi_window, short_name = "RSI").real.to_numpy() # talib got problem calculating RSI?
    fast = vbt.talib("MA").run(data, timeperiod=fast_window, short_name="fast").real.to_numpy()
    slow = vbt.talib("MA").run(data, timeperiod=slow_window, short_name="slow").real.to_numpy()
    entries = (fast > slow) & (rsi < top)
    exits = (fast < slow) & (rsi > bottom)
    return entries, exits

entries, exits = get_signals(data, fast_window, slow_window, 
                             rsi_window, top, bottom)
pf = vbt.Portfolio.from_signals(data, entries, exits, 
                                fees=fees, freq=interval)
pf.total_return()

symbol
BTC-USD    2.254842
ETH-USD    5.437713
Name: total_return, dtype: float64

### Approach 4: Combine indicators using vbt.IndicatorFactory

In [7]:
def combine_rsi_ma(data, fast_window, slow_window, rsi_window, top, bottom):
    rsi = vbt.RSI.run(data, window=rsi_window).rsi.to_numpy()
    fast = vbt.MA.run(data, window=fast_window).ma.to_numpy()
    slow = vbt.MA.run(data, window=slow_window).ma.to_numpy()
    indicator = numpy.where((fast > slow) & (rsi < top), 1, 0)
    indicator = numpy.where((fast < slow) & (rsi > bottom), -1, indicator)
    return indicator

indicator_rsi_ma = vbt.IndicatorFactory(
    class_name='Combination_RSI_MA',
    short_name='RSI_MA',
    input_names=['close'],
    param_names=['fast_window', 'slow_window', 'rsi_window', 'top', 'bottom'],
    output_names=['value'],
).from_apply_func(combine_rsi_ma, fast_window=180, 
                  slow_window=240, rsi_window=14, 
                  top=70, bottom=30)

res = indicator_rsi_ma.run(
    data,
    fast_window=180,
    slow_window=240,
    rsi_window=14,
    top=80,
    bottom=20
)

entries = res.value_above(0) 
exits = res.value_below(0) 

pf = vbt.Portfolio.from_signals(data, entries, exits, 
                                fees=fees, freq=interval)
pf.total_return()

RSI_MA_fast_window  RSI_MA_slow_window  RSI_MA_rsi_window  RSI_MA_top  RSI_MA_bottom  symbol 
180                 240                 14                 80          20             BTC-USD    2.254842
                                                                                      ETH-USD    5.267745
Name: total_return, dtype: float64

### Approach 4: Create custom indicator without built-in indicator but only vbt.IndicatorFactory

In [34]:
# https://vectorbt.dev/api/indicators/factory/#inputs
# https://greyhoundanalytics.com/blog/create-a-custom-indicator-in-vectorbt/
# https://www.youtube.com/watch?v=57hsQz70vVE&t=209s & https://github.com/mikolaje/TradingTutorial/blob/main/VectorBTTutorial/bollinger_band_strategy.ipynb